In [1]:
import pandas as pd
import spacy

In [2]:
#url = 'https://raw.githubusercontent.com/DNason1999/simple_repository/master/df_merged.csv'
url = './Data/best_merge.csv'
df = pd.read_csv(url)

In [3]:
df = df[['id', 'name', 'race', 'Description', 'Rating', 'flavors', 'positive', 'negative', 'medical']]

In [4]:
target = list(df.index)

In [5]:
df = df.set_index('id')

In [6]:
df.shape

(2826, 8)

In [7]:
texts = []

for index, row in df.iterrows():
    s = ''
    for item in row[1:4]:
        if(type(item) == str):
            s += item
        elif(type(item) == float):
            s += str(item)
        s += ' '
    for item in row[6:]:
        if type(item) == float:
            pass
        else:
            for subitem in eval(item):
                s += str(subitem)
                s+= ' '
            s += ' '
    s = s.replace('/',' ').replace('  ', ' ')
                
    texts.append(s)

In [8]:
df['text'] = texts

In [11]:
from api import Tokenizer__

In [12]:
t = Tokenizer__()

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', 
                        ngram_range=(1,2),
                        max_df=.97,
                        min_df=3,
                        tokenizer=t.tokenize)

dtm = tfidf.fit_transform(df['text'])

In [14]:
dtm.shape

(2826, 9432)

In [15]:
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

In [16]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')
nn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [20]:
import joblib

joblib.dump(nn, './Models/nn.pkl', compress=1)

['./Models/nn.pkl']

In [26]:
joblib.dump(tfidf, './Models/tfidf.pkl', compress=1)

['./Models/tfidf.pkl']

In [25]:
# import pickle

# pickle.dump(tfidf, open('./Models/tfidf.pkl', 'wb'))

# pickle.dump(nn, open('./Models/nn.pkl', 'wb'))

In [47]:
sample_text = """
I experience chronic widespread pain that causes my joints, skin, head, and digestive system to hurt, so I need strains that provide significant pain relief. I would also like to find a sleep aid for my insomnia. Depression makes it difficult to wake and get things done, and occasionally, depressive episodes interfere greatly with my function. 
"""

In [48]:
sample_text_tokens = tfidf.transform([sample_text]).todense()

In [49]:
sample_text_tokens

matrix([[0.13333919, 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ]])

In [50]:
predictions = nn.kneighbors(sample_text_tokens)[1][0]

In [51]:
strains = df.iloc[predictions]

In [ ]:
strains